## Импорт и определение библиотек

In [1]:
# отключение параллелизма
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
# ставим разные пакеты оттуда
!pip install langchain langchain-openai langchain-community openai tiktoken langchain-huggingface -q
!pip install beautifulsoup4 pypdf sentence-transformers faiss-cpu unstructured pypandoc -q
#huggingface_hub  langchain_experimental langchainhub
#!pip install unstructured "unstructured[pdf]" -q
!pip install sentence-transformers numpy transformers requests -q

In [3]:
# импортируем библиотеки
from langchain.document_loaders import WebBaseLoader, PyPDFLoader, UnstructuredEPubLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
#from langchain.tools.retriever import create_retriever_tool
#from langchain.agents import initialize_agent, AgentType
from langchain.agents import Tool, initialize_agent
from langchain.prompts import PromptTemplate
import tqdm as notebook_tqdm
from huggingface_hub import HfApi

USER_AGENT environment variable not set, consider setting it to identify your requests.
/home/elijah/Documents/LLM/cocktail_qr/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# для использования ключа из LLM_bot
!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py -q

# Готовим коктейль с помощью RAG
## Чутка `embeddings` с `HuggingFace` для оснастки

In [9]:
# Чтение токена из файла
def read_token(file_path):
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith("HUGGINGFACEHUB_API_TOKEN"):
                return line.split('=')[1].strip()
    raise ValueError("Token not found in the specified file.")

# Загрузка токена
token = read_token('../utils.key')

# Формирование эмбеддингов
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",  
    #"sentence-transformers/paraphrase-multilingual-mpnet-base-v2", 
    #"cointegrated/LaBSE-en-ru",
    model_kwargs={
        "device": "cpu",
        "token": token
                  } 
)

/tmp/ipykernel_53102/634300143.py:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings_model = HuggingFaceEmbeddings(


## Можно пропустить (если используете наполненную чашу векторного хранилища)
### Немного `Document Loader`

In [21]:
# Загрузка EPUB файла от бармена-литературоведа
loader = UnstructuredEPubLoader('../docs/Federle.epub', show_progress=True)
doc_epub = loader.load()

# смотрим, скока получилось загрузить
print('pages count:', len(doc_epub))
print('max chars on page:', max([len(page.page_content) for page in doc_epub]))

pages count: 1
max chars on page: 107261


In [4]:
# Загружаем книгу от крутого бармена
#loader = WebBaseLoader(url)
loader = PyPDFLoader('../docs/Bortnik_1000.pdf')
data = loader.load()

print('pages count:', len(data))
print('max chars on page:', max([len(page.page_content) for page in data]))

# на тот случай, если решим загружать подобную литературу пачкой pdf
"""
#!pip install unstructured "unstructured[pdf]" -q
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader("../docs", glob="**/*.pdf", show_progress=True)
data = loader.load()"""

pages count: 481
max chars on page: 1854


'\n#!pip install unstructured "unstructured[pdf]" -q\nfrom langchain_community.document_loaders import DirectoryLoader\nloader = DirectoryLoader("../docs", glob="**/*.pdf", show_progress=True)\ndata = loader.load()'

### Исполним `Text splitters` и разобьём на чанки

In [5]:
# для gpt-3.5-turbo
splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n', '\n', ' ', ''],  # Разделители для сохранения структуры
    chunk_size=1000,  # 1000 символов (~200-250 токенов)
    chunk_overlap=200  # 200 символов перекрытия
)

# для gpt-4o-mini
"""splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n', '\n', ' ', ''],
    chunk_size=2000,  # 2000 символов (~400-500 токенов)
    chunk_overlap=300  # 300 символов перекрытия
)"""

# Разделяем книгу на части
#documents = splitter.split_documents(doc_epub)

split_documents = []
for page in data:
    split_documents += splitter.create_documents([page.page_content], metadatas=[page.metadata])

len(split_documents)

807

In [7]:
chunks = splitter.split_documents(data)
print('Количество чанков:', len(chunks))
print('Максимальное количество символов в чанке:', max([len(chunk.page_content) for chunk in chunks]))

Количество чанков: 807
Максимальное количество символов в чанке: 1000


### Наполним чашу `Vector Store`

In [10]:
# Если нужно сохранить новый документ в базу
try:
    db_embed = FAISS.from_documents(
            split_documents, 
            hf_embeddings_model
        )
except Exception as e:
    raise RuntimeError(f"Ошибка при создании базы данных FAISS: {e}")


# Сохраняем векторную базу локально
db_embed.save_local("../data/faiss_db_pdf")

## Нужно запустить (если используете готовое векторное хранилище)
### Испьём чашу `Vector Store`

In [13]:
# Если нужно изъять из уже сохранённого документа из базы
# Загрузка векторной базы данных из локального файла
db_embed = FAISS.load_local(
        "../data/faiss_db_pdf", 
        hf_embeddings_model,
        allow_dangerous_deserialization = True
    )

In [17]:
query = "curasao, яблочный ликер"
results = db_embed.similarity_search(query, k=5)
for result in results:
    print(result.page_content)

чавки, хорошо подходит для смешивания коктейлей, так 
как при создании много-
компонентного напитка 
горький привкус исчезает.
Curasao — это общее наз-
вание для ликеров на основе 
кожуры одного из видов 
померанцев. Раньше этот 
вид растений рос преимуще-
ственно на острове Кюра-
сао недалеко от побережья 
Венесуэлы, откуда и произо-
шло его название. В про-
цессе изготовления напитка 
Лимонный ликер.  Этот 
ликер с фруктовым аро-
матом производится из 
спелых плодов лимона.По-
метка «Triple» или «Triple 
Sec» означает макси-
мальное содержание спир-
та 35 %. Как правило, 
крепость этого ликера не 
превышает 30 %.
ПРИЛОЖЕНИЕВИНО
364
АБСЕНТВИСКИВОДКАДЖИНКОНЪЯКРОМ ТЕКИЛАЛИКЕР
сухую кожуру обрабатывают коньяком 
или водкой для получения ароматической 
вытяжки, после чего добавляют пряности 
и травы. Ликер кюрасао бывает разных 
цветов: светло-желтого («Curasao Triple 
Sec»), оранжевого ( «Red Curasao» ), зеле-
ного или синего ( «Curasao Вlue»), а также 
белого («White Curasao»). Обычный 

## Полирнём `Retriver` и инициализирует `llm` для вкуса

In [18]:
# Если используете ключ из курса по LLM, запустите эту ячейку
from utils import ChatOpenAI

# Считываем API ключ из файла
with open('../utils.key', 'r') as file:
    for line in file:
        if line.startswith('OPENAI_API_KEY'):
            course_api_key = line.split('=')[1].strip()
            break

# Инициализация языковой модели
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",         #"gpt-4o-mini", 
    temperature=0.2, 
    course_api_key=course_api_key
)

# объявление retriever 
retriever = db_embed.as_retriever(
        search_type="mmr",              # тип поиска похожих документов
        search_kwargs={
                "k": 5,                 # количество возвращаемых документов (Default: 4)
                "fetch_k": 20
            },     
    )


retrieval_qa = RetrievalQA.from_chain_type(
        llm=llm, 
        retriever=retriever,
        return_source_documents=True    # Возвращает исходные документы
    )

# Месье системный промт

In [19]:
# Системный промт для агента

system_prompt = """
        Ты — помощник для поиска рецептов коктейлей. У тебя есть доступ к базе данных с рецептами, вдохновлёнными литературными произведениями.
        Когда пользователь вводит ингредиенты, используй инструмент "Cocktail Recipe Finder", чтобы найти подходящие рецепты.
        Если рецепт не найден, предложи альтернативные варианты или уточни запрос.

        ===
        
        Пример:
        Пользователь: Какие коктейли можно сделать из водки и апельсинового сока?
        Агент: Использую инструмент "Cocktail Recipe Finder" для поиска рецептов. Вот что найдено: [рецепт].

        ===

        Пример корректного ответа по введённым ингредиентам (ржаной виски, грейпфрутовый сок):
        Коктейль "Рожь и предубеждение"
        Состав:
        - Ржаной виски (50 мл)
        - Грейпфрутовый сок (90 мл)
        Способ приготовления:
        - Ингредиенты выливаем в стакан рокс поверх кубиков льда
    """

# Миксуем рецепт коктейля `agent` 00х

In [20]:
# Создание инструмента
agent_tools = [
    Tool(
        name="Cocktail Recipe Finder",
        func=retrieval_qa.run,
        description="Используй этот инструмент, чтобы найти рецепты коктейлей по заданным ингредиентам. Вводи список ингредиентов, и инструмент вернёт подходящие рецепты. Пример: ['сок', 'виски']."
    )
]

# создание агента
agent = initialize_agent(
    tools=agent_tools, 
    llm=llm, 
    agent="conversational-react-description",       #"zero-shot-react-description", "plan-and-execute"
    verbose=True,
    system_prompt=system_prompt                     # не забыть указать системный промт для вкуса
)

/tmp/ipykernel_53102/3865868748.py:11: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


# Задаём общие параметры, после чего коктейль готов
## Вначале коктейль

In [21]:
# Объявление функции на ввод данных от пользователя
def get_user_input():
    """
    Запрашивает у пользователя ингредиенты для поиска рецептов.
    """
    user_input = input("Введите ингредиенты, разделенные запятыми: ")
    ingredients = [ingredient.strip() for ingredient in user_input.split(",")]
    return ingredients


# Обработка ответа
def format_response(response):
    if "не знаю" in response.lower() or "не найдено" in response.lower():
        return "К сожалению, я не нашёл коктейлей с этими ингредиентами. Попробуйте другие ингредиенты."
    else:
        return response
    

# Функция выбора стиля ответа
def choose_style(styles):
    chosen_style = input("Выберите номер стиля (по умолчанию 4): ") or "4"
    return styles.get(chosen_style, "по-умолчанию")


In [22]:
def main():
    # Определение стилей
    styles = {
        "1": "стиль космического ужаса и хтонического мрака Говарда Ф. Лавкрафта",
        "2": "гопническо-быдляцкий жаргон",
        "3": "экспериментальный стиль нарезок Уильяма Берроуза",
        "4": "стиль барного меню"  # Стиль по умолчанию
    }
    
    while True:
        ingredients = get_user_input()
        if not ingredients:
            print("Вы не ввели ингредиенты. Попробуйте снова.")
            continue
        
        # Преобразуем массив ингредиентов в строку
        ingredients_str = ", ".join(ingredients)
        
        # Формируем текстовый запрос
        user_prompt = (
            """Напиши, какие коктейли можно изготовить из представленных ниже ингредиентов. 
            Для каждого коктейля опиши подробный способ приготовления и укажи пропорции. 
            Отвечай строго на русском языке, используя чёткий и лаконичный формат. 
            Не добавляй лишнюю информацию, которая не относится к рецепту. 
            Ингредиенты: {ingredients}""".format(ingredients=ingredients_str)
        )
        response = retrieval_qa.invoke({"query": user_prompt})  
        
        # Получаем ответ без вывода его на экран
        original_response = response["result"]
        # Выбор стиля
        style = choose_style(styles)
        prompt = PromptTemplate(input_variables=['output_text', 'style'],
                                template='''Добро пожаловать в поиск коктейлей!\nПерепиши этот текст в заданном стиле: {output_text}\nСтиль: {style}.\nРезультат:\n''')
        
        # Генерация текста с использованием оригинального ответа и выбранного стиля
        styled_prompt = prompt.format(output_text=original_response, style=style)
        
        # Применение выбранного стиля к оригинальному ответу
        styled_response = llm.invoke(styled_prompt).content
        print(styled_response)
        break
    
    return styled_response

if __name__ == "__main__":
    # Сохраняем результат в переменную
    recipe = main()


Да вот из этих ингредиентов можно замутить коктейль "Яблочный курасао", братаны!

Ну вот, надо:
- 30 мл яблочного ликера
- 10 мл курасао

Как его делать:
1. В шейкер заливаем яблочный ликер и курасао.
2. Лед добавляем и взбиваем как следует.
3. В коктейльное стекло процеживаем.

Подавать этот коктейль "Яблочный курасао" можно с кусочком яблока на краю стекла. Да будет праздник, братва!
